<a href="https://colab.research.google.com/github/vzaisun/Intrusion-detection-system-using-federated-learning/blob/main/TFF_for_IEC_Using_other_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

'2.14.1'

In [2]:
!pip install tensorflow-federated


In [3]:
!pip install nest-asyncio
!pip install pandas
!pip install matplotlib

In [4]:
import collections
import os
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
import time

from matplotlib import pyplot as plt

import nest_asyncio
nest_asyncio.apply()

%matplotlib inline


In [5]:
n_workers = 2

In [6]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

In [7]:
df_train.dtypes

Flow Duration        int64
Tot Fwd Pkts         int64
Tot Bwd Pkts         int64
TotLen Fwd Pkts      int64
TotLen Bwd Pkts      int64
                    ...   
Idle Mean          float64
Idle Std           float64
Idle Max             int64
Idle Min             int64
Label               object
Length: 77, dtype: object

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3588 entries, 0 to 3587
Data columns (total 77 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      3588 non-null   int64  
 1   Tot Fwd Pkts       3588 non-null   int64  
 2   Tot Bwd Pkts       3588 non-null   int64  
 3   TotLen Fwd Pkts    3588 non-null   int64  
 4   TotLen Bwd Pkts    3588 non-null   int64  
 5   Fwd Pkt Len Max    3588 non-null   int64  
 6   Fwd Pkt Len Min    3588 non-null   int64  
 7   Fwd Pkt Len Mean   3588 non-null   float64
 8   Fwd Pkt Len Std    3588 non-null   float64
 9   Bwd Pkt Len Max    3588 non-null   int64  
 10  Bwd Pkt Len Min    3588 non-null   int64  
 11  Bwd Pkt Len Mean   3588 non-null   float64
 12  Bwd Pkt Len Std    3588 non-null   float64
 13  Flow Byts/s        3588 non-null   float64
 14  Flow Pkts/s        3588 non-null   float64
 15  Flow IAT Mean      3588 non-null   float64
 16  Flow IAT Std       3588 

In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1536 entries, 0 to 1535
Data columns (total 77 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      1536 non-null   int64  
 1   Tot Fwd Pkts       1536 non-null   int64  
 2   Tot Bwd Pkts       1536 non-null   int64  
 3   TotLen Fwd Pkts    1536 non-null   int64  
 4   TotLen Bwd Pkts    1536 non-null   int64  
 5   Fwd Pkt Len Max    1536 non-null   int64  
 6   Fwd Pkt Len Min    1536 non-null   int64  
 7   Fwd Pkt Len Mean   1536 non-null   float64
 8   Fwd Pkt Len Std    1536 non-null   float64
 9   Bwd Pkt Len Max    1536 non-null   int64  
 10  Bwd Pkt Len Min    1536 non-null   int64  
 11  Bwd Pkt Len Mean   1536 non-null   float64
 12  Bwd Pkt Len Std    1536 non-null   float64
 13  Flow Byts/s        1536 non-null   float64
 14  Flow Pkts/s        1536 non-null   float64
 15  Flow IAT Mean      1536 non-null   float64
 16  Flow IAT Std       1536 

In [10]:
df_train['Label'] = df_train['Label'].str.lower()
df_test['Label'] = df_test['Label'].str.lower()

In [11]:
unique_labels = list(df_train.Label.astype('category').unique())
unique_codes = list(df_train.Label.astype('category').cat.codes.unique())
mapping = {unique_codes[i] : unique_labels[i] for i in range(len(unique_labels))}
mapping



{11: 'normal',
 0: 'c_ci_na_1',
 1: 'c_ci_na_1_dos',
 2: 'c_rd_na_1',
 3: 'c_rd_na_1_dos',
 4: 'c_rp_na_1',
 5: 'c_rp_na_1_dos',
 6: 'c_sc_na_1',
 7: 'c_sc_na_1_dos',
 8: 'c_se_na_1',
 9: 'c_se_na_1_dos',
 10: 'm_sp_na_1_dos'}

In [12]:
unique_codes

[11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [13]:
unique_labels

['normal',
 'c_ci_na_1',
 'c_ci_na_1_dos',
 'c_rd_na_1',
 'c_rd_na_1_dos',
 'c_rp_na_1',
 'c_rp_na_1_dos',
 'c_sc_na_1',
 'c_sc_na_1_dos',
 'c_se_na_1',
 'c_se_na_1_dos',
 'm_sp_na_1_dos']

In [14]:
n=len(unique_codes)
for i in range(0,n):
  if(unique_codes[i]!=11):
    unique_codes[i]=1
  else:
    unique_codes[i]=0
unique_codes
#0 is normal, 1 is attack

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [15]:
df_train['Label'] = df_train['Label'].replace(unique_labels, unique_codes)
df_test['Label'] = df_test['Label'].replace(unique_labels, unique_codes)

In [16]:
train = df_train
test = df_test
test

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,179662623,17,18,112,184,16,6,6.588235,2.425356,32,...,0,1.521022e+06,1.832087e+06,3728995,366,1.345083e+07,6.669294e+06,20335549,5996814.0,0
1,179674986,17,18,112,184,16,6,6.588235,2.425356,32,...,0,1.539348e+06,1.855896e+06,3782615,1321,1.343326e+07,6.874232e+06,20324527,5999212.0,0
2,179678255,16,16,96,136,6,6,6.000000,0.000000,16,...,0,1.179574e+06,1.635452e+06,3243750,438,1.515475e+07,7.259378e+06,20437536,5996295.0,0
3,161832605,15,15,90,130,6,6,6.000000,0.000000,16,...,0,1.683661e+06,2.261315e+06,5837467,269,1.449957e+07,7.318009e+06,20169834,5994329.0,0
4,174456115,21,24,136,298,16,6,6.476190,2.182179,36,...,0,2.218205e+06,2.254470e+06,5999865,270,1.206956e+07,7.017426e+06,20684421,5994412.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,179424123,70,360,486,8640,12,6,6.942857,2.199379,32,...,0,0.000000e+00,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0.0,1
1532,179999526,74,361,492,8656,12,6,6.648649,1.875819,32,...,0,0.000000e+00,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0.0,1
1533,179771716,79,360,498,8640,12,6,6.303797,1.323887,32,...,0,0.000000e+00,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0.0,1
1534,89905426,36,182,262,4414,22,6,7.277778,3.282953,48,...,0,0.000000e+00,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0.0,1


In [17]:
n_samples = int(df_train.shape[0] / n_workers)
n_samples_test = int(df_test.shape[0]/n_workers)
assert n_samples > 0, "Each worker must be assigned at least one data point"

In [18]:
import numpy as np
n_epochs = 10
shuffle_buffer_size = df_train.shape[0]
batch_size = 500
prefetch_buffer_size = 100
input_shape = df_train.shape[1]-1
print(input_shape)
output_shape = len(set(unique_codes))
print(output_shape)

76
2


In [19]:
n_epochs = 10
shuffle_buffer_size_test = df_test.shape[0]
print(shuffle_buffer_size_test, "shuffle_buffer_size_test")
batch_size_test = 250
prefetch_buffer_size_test = 50
input_shape_test = df_test.shape[1]-1
print(input_shape_test,"input_shape_test")
output_shape_test = len(set(unique_codes))
print(output_shape_test,"output_shape_test")


1536 shuffle_buffer_size_test
76 input_shape_test
2 output_shape_test


In [20]:
def preprocess(dataframe):

    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    def map_fn(dataset):
        return collections.OrderedDict(
            x=tf.cast(dataset[:,:-1], tf.float64),
            y=tf.cast(tf.reshape(dataset[:,-1], shape=(-1, 1)), tf.int64)
        )

    return tf.data.Dataset.from_tensor_slices(dataframe).repeat(n_epochs).shuffle(
        shuffle_buffer_size).batch(batch_size).map(map_fn).prefetch(prefetch_buffer_size)


client_data = [preprocess(train.sample(n=n_samples)) for _ in range(n_workers)]

In [21]:
def preprocess_test(dataframe):

    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    def map_fn(dataset):
        return collections.OrderedDict(
            x=tf.cast(dataset[:,:-1], tf.float64),
            y=tf.cast(tf.reshape(dataset[:,-1], shape=(-1, 1)), tf.int64)
        )

    return tf.data.Dataset.from_tensor_slices(dataframe).repeat(n_epochs).shuffle(
        shuffle_buffer_size_test).batch(batch_size_test).map(map_fn).prefetch(prefetch_buffer_size_test)


client_data_test = [preprocess(test.sample(n=n_samples_test)) for _ in range(n_workers)]

In [22]:
client_data

[<_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 76), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 76), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>]

In [23]:
client_data_test

[<_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 76), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 76), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>]

In [24]:
for i in range(n_workers):
    print(f"Worker {i+1} data contains {len(client_data[i])} training points")

Worker 1 data contains 36 training points
Worker 2 data contains 36 training points


In [25]:
for i in range(n_workers):
    print(f"Worker {i+1} data contains {len(client_data_test[i])} testing points")

Worker 1 data contains 16 testing points
Worker 2 data contains 16 testing points


In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional, BatchNormalization,Convolution1D,MaxPooling1D, Reshape, GlobalAveragePooling1D,Dense,GroupNormalization
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall

In [27]:
def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Reshape((1, -1)),  # Reshape input to have shape (batch_size, 1, input_dim)
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu')),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    return tff.learning.models.from_keras_model(
        model,
        input_spec=collections.OrderedDict(
            x=tf.TensorSpec(shape=[None, input_shape], dtype=tf.float64),
            y=tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[BinaryAccuracy(), Precision(), Recall(), tf.keras.metrics.SparseCategoricalAccuracy()]
        )

In [28]:
trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01, ema_momentum=0.99),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=1.0)

)

In [29]:
print(trainer.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[76,128],
      float32[32,128],
      float32[128],
      float32[76,128],
      float32[32,128],
      float32[128],
      float32[64,64],
      float32[64],
      float32[64,32],
      float32[32],
      float32[32,1],
      float32[1]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[76,128],
    float32[76,128],
    float32[32,128],
    float32[32,128],
    float32[128],
    float32[128],
    float32[76,128],
    float32[76,128],
    float32[32,128],
    float32[32,128],
    float32[128],
    float32[128],
    float32[64,64],
    float32[64,64],
    float32[64],
    float32[64],
    float32[64,32],
    float32[64,32],
    float32[32],
    float32[32],
    float32[32,1],
    float32[32,1],
    float32[1],
    float32[1]
  >
>@SERVER)


In [36]:
def evaluate(num_rounds):
    state = trainer.initialize()
    for round in range(num_rounds):
        t1 = time.time()
        result = trainer.next(state, client_data)
        state = result.state
        train_metrics = result.metrics['client_work']['train']

        train_accuracy = train_metrics['binary_accuracy'] * 100
        train_precision = train_metrics['precision'] * 100
        train_recall = train_metrics['recall']*100


        t2 = time.time()
        print('Round {} : accuracy {}, Precision {}, Recall {}, round time {}'.format(
            round+1, train_accuracy, train_precision,train_recall, t2 - t1))

In [37]:
evaluate(10)

Round 1 : accuracy 91.02563858032227, Precision 91.20357632637024, Recall 99.78612661361694, round time 1.4930453300476074
Round 2 : accuracy 91.02563858032227, Precision 91.20357632637024, Recall 99.78612661361694, round time 1.3273780345916748
Round 3 : accuracy 91.02563858032227, Precision 91.20357632637024, Recall 99.78612661361694, round time 1.082923412322998
Round 4 : accuracy 91.02563858032227, Precision 91.20357632637024, Recall 99.78612661361694, round time 1.072943925857544
Round 5 : accuracy 91.02563858032227, Precision 91.20357632637024, Recall 99.78612661361694, round time 1.1189155578613281
Round 6 : accuracy 91.02563858032227, Precision 91.20357632637024, Recall 99.78612661361694, round time 1.1557013988494873
Round 7 : accuracy 91.02563858032227, Precision 91.20357632637024, Recall 99.78612661361694, round time 1.0369889736175537
Round 8 : accuracy 91.02563858032227, Precision 91.20357632637024, Recall 99.78612661361694, round time 1.5746073722839355
Round 9 : accuracy

In [38]:
def evaluate_test(num_rounds):
    state = trainer.initialize()
    for round in range(num_rounds):
        t1 = time.time()
        result = trainer.next(state, client_data_test)
        state = result.state
        test_metrics = result.metrics['client_work']['train']

        test_accuracy = test_metrics['binary_accuracy'] * 100
        test_precision = test_metrics['precision'] * 100
        test_recall = test_metrics['recall']*100


        t2 = time.time()
        print('Round {} : accuracy {}, Precision {}, Recall {}, round time {}'.format(
            round+1, test_accuracy, test_precision,test_recall, t2 - t1))

In [39]:
evaluate_test(20)

Round 1 : accuracy 91.53645634651184, Precision 91.6558027267456, Recall 99.85795617103577, round time 0.6880428791046143
Round 2 : accuracy 91.53645634651184, Precision 91.6558027267456, Recall 99.85795617103577, round time 0.7033517360687256
Round 3 : accuracy 91.53645634651184, Precision 91.6558027267456, Recall 99.85795617103577, round time 0.7252769470214844
Round 4 : accuracy 91.53645634651184, Precision 91.6558027267456, Recall 99.85795617103577, round time 0.6971297264099121
Round 5 : accuracy 91.53645634651184, Precision 91.6558027267456, Recall 99.85795617103577, round time 0.6996939182281494
Round 6 : accuracy 91.53645634651184, Precision 91.6558027267456, Recall 99.85795617103577, round time 0.7831366062164307
Round 7 : accuracy 91.53645634651184, Precision 91.6558027267456, Recall 99.85795617103577, round time 0.8079445362091064
Round 8 : accuracy 91.53645634651184, Precision 91.6558027267456, Recall 99.85795617103577, round time 0.780609130859375
Round 9 : accuracy 91.536